In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

---------------

**If any part of this notebook is used in your research, please cite with the reference found in** **[README.md](https://github.com/pysal/spaghetti#bibtex-citation).**


----------------

## Connected components in a spatial network
### Identifying and visualizing the parts of a network

**Author: James D. Gaboardi** **<jgaboardi@gmail.com>**

**This notebook is a walk-through for:**

1. Instantiating a simple network with `libpysal.cg.Chain` objects
2. Working with the network components and isolated rings
3. Visualizing the components and (non)articulation vertices
4. Longest vs. Largest components
5. Extracting network components

In [2]:
%load_ext watermark
%watermark

2020-02-22T21:39:14-05:00

CPython 3.7.3
IPython 7.10.2

compiler   : Clang 9.0.0 (tags/RELEASE_900/final)
system     : Darwin
release    : 19.3.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


<IPython.core.display.Javascript object>

In [3]:
import geopandas
import libpysal
from libpysal import examples
from libpysal.cg import Point, Chain
import matplotlib
import matplotlib_scalebar
from matplotlib_scalebar.scalebar import ScaleBar
import spaghetti

%matplotlib inline
%watermark -w
%watermark -iv

ImportError: cannot import name 'extract_components' from 'spaghetti.network' (/Users/jgaboardi/spaghetti/spaghetti/network.py)

<IPython.core.display.Javascript object>

In [ ]:
try:
    from IPython.display import set_matplotlib_formats
    set_matplotlib_formats("retina")
except ImportError:
    pass

----------------

### 1. Instantiate a network from two collections of `libpysal.cg.Chain` objects

In [ ]:
plus1 = [
    Chain([Point([1, 2]), Point([0, 2])]),
    Chain([Point([1, 2]), Point([1, 1])]),
    Chain([Point([1, 2]), Point([1, 3])]),
]
plus2 = [
    Chain([Point([2, 1]), Point([2, 0])]),
    Chain([Point([2, 1]), Point([3, 1])]),
    Chain([Point([2, 1]), Point([2, 2])]),
]
lines = plus1 + plus2

In [ ]:
ntw = spaghetti.Network(in_data=lines)

#### Here we get a warning because the network we created is not fully connected

In [ ]:
ntw.network_fully_connected

#### It has 2 connected components

In [ ]:
ntw.network_n_components

#### The network components can be inspected through the following attributes
##### `network_component_labels`

In [ ]:
ntw.network_component_labels

##### `network_component2arc` 

In [ ]:
ntw.network_component2arc

##### `network_component_lengths`

In [ ]:
ntw.network_component_lengths

##### `network_longest_component`

In [ ]:
ntw.network_longest_component

##### `network_component_vertices` 

In [ ]:
ntw.network_component_vertices

##### `network_component_vertex_count` 

In [ ]:
ntw.network_component_vertex_count

##### `network_largest_component` 

In [ ]:
ntw.network_largest_component

##### `network_component_is_ring` 

In [ ]:
ntw.network_component_is_ring

#### The same can be performed for graph representations, for example:
##### `graph_component_labels`

In [ ]:
ntw.graph_component_labels

##### `graph_component2edge` 

In [ ]:
ntw.graph_component2edge

#### Extract the network arc and vertices as `geopandas.GeoDataFrame` objects

In [ ]:
# network vertices and arcs
vertices_df, arcs_df = spaghetti.element_as_gdf(ntw, vertices=True, arcs=True)

#### Network component labels are found in the `"comp_label"` field

In [ ]:
vertices_df

In [ ]:
arcs_df

#### Plot the disconnected network and symbolize the arcs bases on the value of `"comp_label"`

In [ ]:
base = arcs_df.plot(column="comp_label", cmap="Set2", linewidth=5, figsize=(7, 7))
vertices_df.plot(ax=base, color="k", markersize=100, zorder=2);


----------------

### 2. Add to the network created above

In [ ]:
new_lines = [
    Chain([Point([1, 1]), Point([2, 2])]),
    Chain([Point([0.5, 1]), Point([0.5, 0.5])]),
    Chain([Point([0.5, 0.5]), Point([1, 0.5])]),
    Chain([Point([2, 2.5]), Point([2.5, 2.5])]),
    Chain([Point([2.5, 2.5]), Point([2.5, 2])]),
]
lines += new_lines

In [ ]:
ntw = spaghetti.Network(in_data=lines)

#### Now there are 3 connected components in the network

In [ ]:
ntw.network_n_components

In [ ]:
ntw.network_component2arc

In [ ]:
# network vertices and arcs
vertices_df, arcs_df = spaghetti.element_as_gdf(ntw, vertices=True, arcs=True)

In [ ]:
arcs_df

#### We can also inspect the non-articulation points in the network. Non-articulation points are vertices in a network that are degree-2. A vertex is degree-2 if, and only if, it is directly connected to only 2 other vertices.

In [ ]:
ntw.non_articulation_points

#### Slice out the articulation points and non-articulation points

In [ ]:
napts = ntw.non_articulation_points
articulation_vertices = vertices_df[~vertices_df["id"].isin(napts)]
non_articulation_vertices = vertices_df[vertices_df["id"].isin(napts)]

#### Plot the connected components while making a distinction between articulation points and non-articulation points

In [ ]:
base = arcs_df.plot(column="comp_label", cmap="Set2", linewidth=5, figsize=(7, 7))
articulation_vertices.plot(ax=base, color="k", markersize=100, zorder=2)
non_articulation_vertices.plot(ax=base, marker="s", color="k", markersize=20, zorder=2);

----------------

### 3. Add a loop of `libpysal.cg.Chain` objects

In [ ]:
new_lines = [
    Chain([Point([3, 1]), Point([3.25, 1.25])]),
    Chain([Point([3.25, 1.25]), Point([3.5, 1.25])]),
    Chain([Point([3.5, 1.25]), Point([3.75, 1])]),
    Chain([Point([3.75, 1]), Point([3.5, .75])]),
    Chain([Point([3.5, .75]), Point([3.25, .75])]),
    Chain([Point([3.25, .75]), Point([3, 1])]),
]
lines += new_lines

In [ ]:
ntw = spaghetti.Network(in_data=lines)

In [ ]:
ntw.network_n_components

In [ ]:
ntw.network_component2arc

In [ ]:
# network vertices and arcs
vertices_df, arcs_df = spaghetti.element_as_gdf(ntw, vertices=True, arcs=True)

In [ ]:
arcs_df

#### Here we can see that all the new network vertices are non-articulation point

In [ ]:
ntw.non_articulation_points

#### Slice out the articulation points and non-articulation points

In [ ]:
napts = ntw.non_articulation_points
articulation_vertices = vertices_df[~vertices_df["id"].isin(napts)]
non_articulation_vertices = vertices_df[vertices_df["id"].isin(napts)]

#### The new network vertices are non-articulation points because they form a closed ring

In [ ]:
base = arcs_df.plot(column="comp_label", cmap="Set2", linewidth=5, figsize=(7, 7))
articulation_vertices.plot(ax=base, color="k", markersize=100, zorder=2)
non_articulation_vertices.plot(ax=base, marker="s", color="k", markersize=20, zorder=2);

-----------------------------------------------------

### 4. Longest vs. largest components — cross vs. hexagon

In [ ]:
cross = [
    Chain([Point([0, 5]), Point([5, 5]), Point([5, 10])]),
    Chain([Point([5, 0]), Point([5, 5]), Point([10, 5])])
]
hexagon = [
    Chain(
        [
            Point([12, 5]),
            Point([13, 6]),
            Point([14, 6]),
            Point([15, 5]),
            Point([14, 4]),
            Point([13, 4]),
            Point([12, 5])
        ]
    ),
]
lines = cross + hexagon

In [ ]:
ntw = spaghetti.Network(in_data=lines)

In [ ]:
# network vertices and arcs
vertices_df, arcs_df = spaghetti.element_as_gdf(ntw, vertices=True, arcs=True)

In [ ]:
base = arcs_df.plot(column="comp_label", cmap="Set2", linewidth=5, figsize=(7, 7))
vertices_df.plot(ax=base, color="k", markersize=100, zorder=2);

#### The longest component is not necessarily the largest
##### This is because in `spaghetti` the largest compnent equates to the most vertices

In [ ]:
ntw.network_longest_component, ntw.network_largest_component

### 5. Extracting components
#### Extract the longest component

In [ ]:
longest = spaghetti.extract_component(ntw, ntw.network_longest_component)

In [ ]:
# network vertices and arcs
vertices_df, arcs_df = spaghetti.element_as_gdf(longest, vertices=True, arcs=True)

In [ ]:
vertices_df

In [ ]:
base = arcs_df.plot(column="comp_label", cmap="Set2", linewidth=5, figsize=(7, 7))
vertices_df.plot(ax=base, color="k", markersize=100, zorder=2);

#### Extract the largest component and plot

In [ ]:
largest = spaghetti.extract_component(ntw, ntw.network_largest_component)
# network vertices and arcs
vertices_df, arcs_df = spaghetti.element_as_gdf(largest, vertices=True, arcs=True)
base = arcs_df.plot(column="comp_label", cmap="Set2", linewidth=5, figsize=(7, 7))
vertices_df.plot(ax=base, color="k", markersize=100, zorder=2);

#### Empirical Example — New Haven, Connecticut

In [ ]:
newhaven = libpysal.examples.get_path("newhaven_nework.shp")
ntw = spaghetti.Network(in_data=newhaven, extractgraph=False)

#### Extract the longest component

In [ ]:
longest = spaghetti.extract_component(ntw, ntw.network_longest_component)

In [ ]:
# network vertices and arcs
vertices_df, arcs_df = spaghetti.element_as_gdf(ntw, vertices=True, arcs=True)
arcs_df.crs = "epsg:4269"
arcs_df = arcs_df.to_crs("epsg:6433")

In [ ]:
# longest vertices and arcs
lc_vertices, lc_arcs = spaghetti.element_as_gdf(longest, vertices=True, arcs=True)
lc_arcs.crs = "epsg:4269"
lc_arcs = lc_arcs.to_crs("epsg:6433")

#### Filter non-longest component arcs

In [ ]:
nlc = ntw.network_longest_component
arcs_df = arcs_df[arcs_df.comp_label != nlc]
ocomp = list(set(ntw.network_component_labels))
ocomp.remove(nlc)

#### Plot network arcs

In [ ]:
def legend(objects):
    """Add a legend to a plot"""
    patches = make_patches(*objects)
    kws = {"fancybox": True, "framealpha": 0.85, "fontsize": "x-large"}
    kws.update({"loc":"lower left", "labelspacing":2., "borderpad":2.})
    legend = matplotlib.pyplot.legend(handles=patches, **kws)
    legend.get_frame().set_facecolor("white")

In [ ]:
def make_patches(comp_type, in_comp, oc):
    """Create patches for legend"""
    labels_colors_alpha = [
        ["%s component: %s" % (comp_type.capitalize(), in_comp), "k", .5], 
        ["Other components: %s-%s" % (oc[0], oc[1]), "r", 1]
    ]
    patches = []
    for l, c, a in labels_colors_alpha:
        p = matplotlib.lines.Line2D([], [], lw=2, label=l, c=c, alpha=a)
        patches.append(p)
    return patches

In [ ]:
base = arcs_df.plot(color="r", alpha=1, linewidth=3, figsize=(10, 10))
lc_arcs.plot(ax=base, color="k", linewidth=2, alpha=.5, zorder=2);
# add legend
legend(("longest", nlc, (ocomp[0], ocomp[-1])))
# add scale bar
scalebar = ScaleBar(3, units="m", location="lower right")
base.add_artist(scalebar);

---------------------------------------